In [1]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', 70)
import matplotlib.pyplot as plt

df=pd.read_csv("new\Heat_Flux_Data_New.csv")
dti=pd.date_range('00:00:00', periods= len(df), freq= "1S")
df.set_index(dti, inplace=True)

#given params for Buckets
mold_w=[[1100, 1150],[1150,1200],[1200, 1250],[1250,1300],[1300,1350],[1350,1400],[1400,1450],[1450,1500],[1500,1550],[1550,1600],[1600,1700]]
thick_BD= [[25.40, 27],[23.80, 25.39],[22.20, 23.79], [20.60, 22.59],[19.00, 20.59],[17.40, 18.99]]
thick_NW= [[18,19],[17,18],[16,17],[15,16],[14,15],[13,14]]

In [2]:
#Function to Print Range for given Bucket on basis of Mean, Median, Mid-Point

def get_ranges(s, name):
    
    attrib= s.columns[0]
    
    #assign cutoff (1.2 for heat_flux, 80 for heat_flux_ratio)
    if len(attrib)==11:
        cutoff=1.2
    else:
        cutoff=60
        
    #mean_calculations
    temp=pd.DataFrame()
    dist=pd.DataFrame()
    df_mean=pd.DataFrame()
    temp=s[s>cutoff]
    temp = temp[temp[attrib].notna()]
    mean__= temp.mean().item()
    dist['abs_dist']= np.abs(temp - temp.mean()).sum(axis=1).to_frame()
    dist['relative_dist']= (temp - temp.mean()).sum(axis=1).to_frame()

    percentage_set=[50, 70, 95]

    min_set_mean=[]
    max_set_mean=[]


    for percentage in [50, 70, 95]:

        df_mean=pd.DataFrame()
        end_limit= int((percentage/100)* len(temp))
        df_mean= dist.sort_values(by='abs_dist')[:end_limit]
        
        min_location= df_mean[df_mean['relative_dist']==df_mean['relative_dist'].min()].index[0]
        min_mean= temp[temp.index==min_location].min().item()

        max_location= df_mean[df_mean['relative_dist']==df_mean['relative_dist'].max()].index[0]
        max_mean= temp[temp.index==max_location].max().item()
        
        min_set_mean.append(float(min_mean))
        max_set_mean.append(float(max_mean))

    
    
    #change dataframe to series    
    s=s.squeeze()
    s=pd.Series(s)
        
    #take only values above cutoff
    s=s[s>cutoff]
    
    percentage_set=[25, 50, 75]
        
    #Median Calculations
    min_set_median=[]
    max_set_median=[]
    
    
    mid_no= len(s[s<s.median()])

    for percent in percentage_set:
        lower_bound= int(mid_no- ((percent/100)* mid_no))
        upper_bound= int(mid_no+ ((percent/100) * mid_no))
        min_= s.sort_values()[lower_bound: mid_no].min()
        max_= s.sort_values()[mid_no: upper_bound].max()
        
        min_set_median.append(min_)
        max_set_median.append(max_)
        
    #MidPoint Calculations
    
    min_set_mid=[]
    max_set_mid=[]
    
    mid_point= s.min()+ ((s.max()-s.min())/2)
    mid_point= round(mid_point,2)
    
    for percent in percentage_set:
        lb= mid_point - ((percent/100) * (mid_point-s.min()))
        ub= mid_point + ((percent/100) * (s.max()-mid_point))      
        min_set_mid.append(round(lb,2))
        max_set_mid.append(round(ub,2))
        
    #Mean calculations
        
    
  
   
    return [name, s.max(), s.min(), round(mean__,2), s.median() , mid_point, min_set_mean[0], max_set_mean[0], min_set_mean[1], max_set_mean[1], min_set_mean[2], max_set_mean[2],min_set_median[0], max_set_median[0], min_set_median[1], max_set_median[1], min_set_median[2], max_set_median[2], min_set_mid[0], max_set_mid[0], min_set_mid[1], max_set_mid[1],min_set_mid[2], max_set_mid[2]]
    
    


In [3]:
thickness=[55,60]
cast_speed=5.5

#Creating Buckets and naming them by their lowerlimits eg vars()['1300x15']

bucket_names=[]
num1=len(mold_w)
num2=len(thick_BD)
for i in range(num1):
    for j in range(num2):
        title= str(mold_w[i][0])+ 'x' + str(thick_NW[j][0])
        bucket_names.append(title)
        vars()[str(title)]=df.loc[((df['CASTSPEED'] > cast_speed) & (df['SLAB_THICKNESS'] >= thickness[0]) & (df['SLAB_THICKNESS'] <= thickness[1]) & (df['MOLD_WIDTH'] > mold_w[i][0]) & (df['MOLD_WIDTH'] <= mold_w[i][1]) & (df['PLT_THK_NSRS'] > thick_NW[j][0]) & (df['PLT_THK_NSRS'] <= thick_NW[j][1]) & (df['PLT_THK_LS'] > thick_BD[j][0]) & (df['PLT_THK_LS'] <= thick_BD[j][1]))]

In [4]:
#print bucket params

com=[]
s=0

for i in range(len(mold_w)):
    for j in range(len(thick_BD)):
        s=s+1
        com.append([str(cast_speed),'7' ,str(thick_BD[j][0]),str(thick_BD[j][1]), str(thick_NW[j][0]),str(thick_NW[j][1]),str(mold_w[i][0]),str(mold_w[i][1]),str(thickness[0]),str(thickness[1])])

template_df = pd.DataFrame(com, columns =['CAST_SPEED_MIN', 'CAST_SPEED_MAX', 'PLT_THK_LS_MIN', 'PLT_THK_LS_MAX', 'PLT_THK_NSRS_MIN', 'PLT_THK_NSRS_MAX', 'WIDTH_MIN', 'WIDTH_MAX', 'THICK_MIN', 'THICK_MAX'])

template_df['HF_LEFT_MIN_95'] = " "
template_df['HF_LEFT_MAX_95'] = " "
template_df['HF_LEFT_MIN_70'] = " "
template_df['HF_LEFT_MAX_70'] = " "
template_df['HF_RIGHT_MIN_95'] = " "
template_df['HF_RIGHT_MAX_95'] = " "
template_df['HF_RIGHT_MIN_70'] = " "
template_df['HF_RIGHT_MAX_70'] = " "
template_df['HFR_LEFT_MIN_95'] = " "
template_df['HFR_LEFT_MAX_95'] = " "
template_df['HFR_LEFT_MIN_70'] = " "
template_df['HFR_LEFT_MAX_70'] = " "
template_df['HFR_RIGHT_MIN_95'] = " "
template_df['HFR_RIGHT_MAX_95'] = " "
template_df['HFR_RIGHT_MIN_70'] = " "
template_df['HFR_RIGHT_MAX_70'] = " "
template_df['HF_FIX_MIN_95'] = " "
template_df['HF_FIX_MAX_95'] = " "
template_df['HF_FIX_MIN_70'] = " "
template_df['HF_FIX_MAX_70'] = " "
template_df['HF_LOOSE_MIN_95'] = " "
template_df['HF_LOOSE_MAX_95'] = " "
template_df['HF_LOOSE_MIN_70'] = " "
template_df['HF_LOOSE_MAX_70'] = " "


In [5]:
range_attribute='HEAT_FLUX_L'

complete_set=[]
for name in bucket_names:
    s = vars()[name][range_attribute].to_frame()
    if len(s)>25:
         complete_set.append(get_ranges(s, name))
range_hf_l = pd.DataFrame(complete_set, columns =['BucketName', 'Max','Min', 'Mean', 'Median', 'Mid_Point (Max-Min)',  '50% Below Mean','50% Above Mean','70% Below Mean','70% Above Mean','95% Below Mean','95% Above Mean','25% Below Median','25% Above Median','50% Below Median','50% Above Median','75% Below Median','75% Above Median', '25% Below MidPoint','25% Above MidPoint','50% Below MidPoint','50% Above MidPoint','75% Below MidPoint','75%Points Above MidPoint'])

for i in range(len(range_hf_l)):
    width_c = range_hf_l['BucketName'].iloc[i][:4]
    plt_c=  range_hf_l['BucketName'].iloc[i][-2:]
    
    for j in range(len(template_df)):
        if template_df['WIDTH_MIN'].iloc[j]==width_c and template_df['PLT_THK_NSRS_MIN'].iloc[j]==plt_c:
            template_df['HF_LEFT_MIN_95'].iloc[j]= range_hf_l['95% Below Mean'].iloc[i]
            template_df['HF_LEFT_MAX_95'].iloc[j]= range_hf_l['95% Above Mean'].iloc[i]
            template_df['HF_LEFT_MIN_70'].iloc[j]= range_hf_l['70% Below Mean'].iloc[i]
            template_df['HF_LEFT_MAX_70'].iloc[j]= range_hf_l['70% Above Mean'].iloc[i]

In [6]:
#Create DataFrame

range_attribute='HEAT_FLUX_R'

complete_set=[]
for name in bucket_names:
    s = vars()[name][range_attribute].to_frame()
    if len(s)>20:
         complete_set.append(get_ranges(s, name))
range_hf_r = pd.DataFrame(complete_set, columns =['BucketName', 'Max','Min', 'Mean', 'Median', 'Mid_Point (Max-Min)',  '50% Below Mean','50% Above Mean','70% Below Mean','70% Above Mean','95% Below Mean','95% Above Mean','25% Below Median','25% Above Median','50% Below Median','50% Above Median','75% Below Median','75% Above Median', '25% Below MidPoint','25% Above MidPoint','50% Below MidPoint','50% Above MidPoint','75% Below MidPoint','75%Points Above MidPoint'])
###

for i in range(len(range_hf_r)):
    width_c = range_hf_r['BucketName'].iloc[i][:4]
    plt_c=  range_hf_r['BucketName'].iloc[i][-2:]
    
    
    for j in range(len(template_df)):
        if template_df['WIDTH_MIN'].iloc[j]==width_c and template_df['PLT_THK_NSRS_MIN'].iloc[j]==plt_c:
            template_df['HF_RIGHT_MIN_95'].iloc[j]= range_hf_r['95% Below Mean'].iloc[i]
            template_df['HF_RIGHT_MAX_95'].iloc[j]= range_hf_r['95% Above Mean'].iloc[i]
            template_df['HF_RIGHT_MIN_70'].iloc[j]= range_hf_r['70% Below Mean'].iloc[i]
            template_df['HF_RIGHT_MAX_70'].iloc[j]= range_hf_r['70% Above Mean'].iloc[i]

In [7]:
range_attribute='HEAT_FLUX_RATIO_L'


complete_set=[]
for name in bucket_names:
    s = vars()[name][range_attribute].to_frame()
    if len(s)>25:
         complete_set.append(get_ranges(s, name))
range_hf_ratio_l = pd.DataFrame(complete_set, columns =['BucketName', 'Max','Min', 'Mean', 'Median', 'Mid_Point (Max-Min)',  '50% Below Mean','50% Above Mean','70% Below Mean','70% Above Mean','95% Below Mean','95% Above Mean','25% Below Median','25% Above Median','50% Below Median','50% Above Median','75% Below Median','75% Above Median', '25% Below MidPoint','25% Above MidPoint','50% Below MidPoint','50% Above MidPoint','75% Below MidPoint','75%Points Above MidPoint'])


for i in range(len(range_hf_ratio_l)):
    width_c = range_hf_ratio_l['BucketName'].iloc[i][:4]
    plt_c=  range_hf_ratio_l['BucketName'].iloc[i][-2:]
    
    
    for j in range(len(template_df)):
        if template_df['WIDTH_MIN'].iloc[j]==width_c and template_df['PLT_THK_NSRS_MIN'].iloc[j]==plt_c:
            template_df['HFR_LEFT_MIN_95'].iloc[j]= range_hf_ratio_l['95% Below Mean'].iloc[i]
            template_df['HFR_LEFT_MAX_95'].iloc[j]= range_hf_ratio_l['95% Above Mean'].iloc[i]
            template_df['HFR_LEFT_MIN_70'].iloc[j]= range_hf_ratio_l['70% Below Mean'].iloc[i]
            template_df['HFR_LEFT_MAX_70'].iloc[j]= range_hf_ratio_l['70% Above Mean'].iloc[i]
            

In [8]:
range_attribute='HEAT_FLUX_RATIO_R'

complete_set=[]
for name in bucket_names:
    s = vars()[name][range_attribute].to_frame()
    if len(s)>25:
         complete_set.append(get_ranges(s, name))
range_hf_ratio_r = pd.DataFrame(complete_set, columns =['BucketName', 'Max','Min', 'Mean', 'Median', 'Mid_Point (Max-Min)',  '50% Below Mean','50% Above Mean','70% Below Mean','70% Above Mean','95% Below Mean','95% Above Mean','25% Below Median','25% Above Median','50% Below Median','50% Above Median','75% Below Median','75% Above Median', '25% Below MidPoint','25% Above MidPoint','50% Below MidPoint','50% Above MidPoint','75% Below MidPoint','75%Points Above MidPoint'])

for i in range(len(range_hf_ratio_r)):
    width_c = range_hf_ratio_r['BucketName'].iloc[i][:4]
    plt_c=  range_hf_ratio_r['BucketName'].iloc[i][-2:]
    
    
    for j in range(len(template_df)):
        if template_df['WIDTH_MIN'].iloc[j]==width_c and template_df['PLT_THK_NSRS_MIN'].iloc[j]==plt_c:
            template_df['HFR_RIGHT_MIN_95'].iloc[j]= range_hf_ratio_r['95% Below Mean'].iloc[i]
            template_df['HFR_RIGHT_MAX_95'].iloc[j]= range_hf_ratio_r['95% Above Mean'].iloc[i]
            template_df['HFR_RIGHT_MIN_70'].iloc[j]= range_hf_ratio_r['70% Below Mean'].iloc[i]
            template_df['HFR_RIGHT_MAX_70'].iloc[j]= range_hf_ratio_r['70% Above Mean'].iloc[i]
            
template_df[10:]

,CAST_SPEED_MIN,CAST_SPEED_MAX,PLT_THK_LS_MIN,PLT_THK_LS_MAX,PLT_THK_NSRS_MIN,PLT_THK_NSRS_MAX,WIDTH_MIN,WIDTH_MAX,THICK_MIN,THICK_MAX,HF_LEFT_MIN_95,HF_LEFT_MAX_95,HF_LEFT_MIN_70,HF_LEFT_MAX_70,HF_RIGHT_MIN_95,HF_RIGHT_MAX_95,HF_RIGHT_MIN_70,HF_RIGHT_MAX_70,HFR_LEFT_MIN_95,HFR_LEFT_MAX_95,HFR_LEFT_MIN_70,HFR_LEFT_MAX_70,HFR_RIGHT_MIN_95,HFR_RIGHT_MAX_95,HFR_RIGHT_MIN_70,HFR_RIGHT_MAX_70,HF_FIX_MIN_95,HF_FIX_MAX_95,HF_FIX_MIN_70,HF_FIX_MAX_70,HF_LOOSE_MIN_95,HF_LOOSE_MAX_95,HF_LOOSE_MIN_70,HF_LOOSE_MAX_70
10,5.5,7,19.0,20.59,14,15,1150,1200,55,60,,,,,,,,,,,,,,,,,,,,,,,,
11,5.5,7,17.4,18.99,13,14,1150,1200,55,60,,,,,,,,,,,,,,,,,,,,,,,,
12,5.5,7,25.4,27,18,19,1200,1250,55,60,,,,,,,,,,,,,,,,,,,,,,,,
13,5.5,7,23.8,25.39,17,18,1200,1250,55,60,,,,,,,,,,,,,,,,,,,,,,,,
14,5.5,7,22.2,23.79,16,17,1200,1250,55,60,,,,,,,,,,,,,,,,,,,,,,,,
15,5.5,7,20.6,22.59,15,16,1200,1250,55,60,,,,,,,,,,,,,,,,,,,,,,,,
16,5.5,7,19.0,20.59,14,15,1200,1250,55,60,,,,,,,,,,,,,,,,,,,,,,,,
17,5.5,7,17.4,18.99,13,14,1200,1250,55,60,,,,,,,,,,,,,,,,,,,,,,,,
18,5.5,7,25.4,27,18,19,1250,1300,55,60,2.16,2.74,2.27,2.64,2.13,2.62,2.23,2.52,81.27,102.32,86.05,97.65,78.298,104.153,85.345,97.158,,,,,,,,
19,5.5,7,23.8,25.39,17,18,1250,1300,55,60,2.04,2.75,2.26,2.52,1.95,2.64,2.12,2.55,82.02,104.01,86.87,98.83,81.133,102.246,86.438,97.419,,,,,,,,


In [9]:
# New Data to calculate Heat_Flux_Fix and Loose again

In [10]:
df=pd.read_csv("new\hEATFLUX_lOOSE_FIX_DATA.csv")
dti=pd.date_range('00:00:00', periods= len(df), freq= "1S")
df.set_index(dti, inplace=True)

#given params for Buckets
mold_w=[[1100, 1150],[1150,1200],[1200, 1250],[1250,1300],[1300,1350],[1350,1400],[1400,1450],[1450,1500],[1500,1550],[1550,1600],[1600,1700]]
thick_BD= [[25.40, 27],[23.80, 25.39],[22.20, 23.79], [20.60, 22.59],[19.00, 20.59],[17.40, 18.99]]
thick_NW= [[18,19],[17,18],[16,17],[15,16],[14,15],[13,14]]

thickness=[55,60]
cast_speed=5.5

def get_loose_fix(s, name):
    
    threshold=1.5
    
    temp=pd.DataFrame()
    temp=s[s>threshold]
    temp=temp.dropna()
    if len(temp)<5:
        
        return name, np.nan, np.nan,  np.nan, np.nan,  np.nan, np.nan,  np.nan
    
    mean__=temp.mean().item()

    max__=temp.max().item()
    min__=temp.min().item()

    dist=pd.DataFrame()
    dist['abs_dist']= np.abs(temp - temp.mean()).sum(axis=1).to_frame()
    dist['relative_dist']= (temp - temp.mean()).sum(axis=1).to_frame()

    min_set_mean=[]
    max_set_mean=[]


    for percentage in [70, 95]:
        df_mean=pd.DataFrame()
        end_limit= int((percentage/100)* len(temp))
        df_mean= dist.sort_values(by='abs_dist')[:end_limit]

        min_location= df_mean[df_mean['relative_dist']==df_mean['relative_dist'].min()].index[0]
        min_mean= temp[temp.index==min_location].min().item()

        max_location= df_mean[df_mean['relative_dist']==df_mean['relative_dist'].max()].index[0]
        max_mean= temp[temp.index==max_location].max().item()

        min_set_mean.append(float(min_mean))
        max_set_mean.append(float(max_mean))

        
    return name, mean__, max__, min__, min_set_mean[0], max_set_mean[0], min_set_mean[1], max_set_mean[1]




In [11]:
range_attribute='HEAT_FLUX_FIX'

complete_set=[]
for bucket in bucket_names:
    complete_set.append(get_loose_fix(vars()[bucket][range_attribute].to_frame(), bucket))
    
    
range_fix = pd.DataFrame(complete_set, columns =['BucketName', 'Mean','Max', 'Min', '70% Below Mean','70% Above Mean','95% Below Mean','95% Above Mean'])
range_fix= range_fix.dropna()

for i in range(len(range_fix)):
    width_c = range_fix['BucketName'].iloc[i][:4]
    plt_c=  range_fix['BucketName'].iloc[i][-2:]
    
    
    for j in range(len(template_df)):
        if template_df['WIDTH_MIN'].iloc[j]==width_c and template_df['PLT_THK_NSRS_MIN'].iloc[j]==plt_c:
            template_df['HF_FIX_MIN_95'].iloc[j]= range_fix['95% Below Mean'].iloc[i]
            template_df['HF_FIX_MAX_95'].iloc[j]= range_fix['95% Above Mean'].iloc[i]
            template_df['HF_FIX_MIN_70'].iloc[j]= range_fix['70% Below Mean'].iloc[i]
            template_df['HF_FIX_MAX_70'].iloc[j]= range_fix['70% Above Mean'].iloc[i]


In [12]:
range_attribute='HEAT_FLUX_LOOSE'

complete_set=[]
for bucket in bucket_names:
    complete_set.append(get_loose_fix(vars()[bucket][range_attribute].to_frame(), bucket))
print('-'*110)
print(' '*25+ 'Ranges for {}'.format(range_attribute)+ ' According to New Data')
print('-'*110)       
range_loose = pd.DataFrame(complete_set, columns =['BucketName', 'Mean','Max', 'Min', '70% Below Mean','70% Above Mean','95% Below Mean','95% Above Mean'])
range_loose= range_loose.dropna()

for i in range(len(range_loose)):
    width_c = range_loose['BucketName'].iloc[i][:4]
    plt_c=  range_loose['BucketName'].iloc[i][-2:]
    
    
    for j in range(len(template_df)):
        if template_df['WIDTH_MIN'].iloc[j]==width_c and template_df['PLT_THK_NSRS_MIN'].iloc[j]==plt_c:
            template_df['HF_LOOSE_MIN_95'].iloc[j]= range_loose['95% Below Mean'].iloc[i]
            template_df['HF_LOOSE_MAX_95'].iloc[j]= range_loose['95% Above Mean'].iloc[i]
            template_df['HF_LOOSE_MIN_70'].iloc[j]= range_loose['70% Below Mean'].iloc[i]
            template_df['HF_LOOSE_MAX_70'].iloc[j]= range_loose['70% Above Mean'].iloc[i]
       
template_df[10:]


--------------------------------------------------------------------------------------------------------------
                         Ranges for HEAT_FLUX_LOOSE According to New Data
--------------------------------------------------------------------------------------------------------------


,CAST_SPEED_MIN,CAST_SPEED_MAX,PLT_THK_LS_MIN,PLT_THK_LS_MAX,PLT_THK_NSRS_MIN,PLT_THK_NSRS_MAX,WIDTH_MIN,WIDTH_MAX,THICK_MIN,THICK_MAX,HF_LEFT_MIN_95,HF_LEFT_MAX_95,HF_LEFT_MIN_70,HF_LEFT_MAX_70,HF_RIGHT_MIN_95,HF_RIGHT_MAX_95,HF_RIGHT_MIN_70,HF_RIGHT_MAX_70,HFR_LEFT_MIN_95,HFR_LEFT_MAX_95,HFR_LEFT_MIN_70,HFR_LEFT_MAX_70,HFR_RIGHT_MIN_95,HFR_RIGHT_MAX_95,HFR_RIGHT_MIN_70,HFR_RIGHT_MAX_70,HF_FIX_MIN_95,HF_FIX_MAX_95,HF_FIX_MIN_70,HF_FIX_MAX_70,HF_LOOSE_MIN_95,HF_LOOSE_MAX_95,HF_LOOSE_MIN_70,HF_LOOSE_MAX_70
10,5.5,7,19.0,20.59,14,15,1150,1200,55,60,,,,,,,,,,,,,,,,,,,,,,,,
11,5.5,7,17.4,18.99,13,14,1150,1200,55,60,,,,,,,,,,,,,,,,,,,,,,,,
12,5.5,7,25.4,27,18,19,1200,1250,55,60,,,,,,,,,,,,,,,,,,,,,,,,
13,5.5,7,23.8,25.39,17,18,1200,1250,55,60,,,,,,,,,,,,,,,,,,,,,,,,
14,5.5,7,22.2,23.79,16,17,1200,1250,55,60,,,,,,,,,,,,,,,,,,,,,,,,
15,5.5,7,20.6,22.59,15,16,1200,1250,55,60,,,,,,,,,,,,,,,,,,,,,,,,
16,5.5,7,19.0,20.59,14,15,1200,1250,55,60,,,,,,,,,,,,,,,,,,,,,,,,
17,5.5,7,17.4,18.99,13,14,1200,1250,55,60,,,,,,,,,,,,,,,,,,,,,,,,
18,5.5,7,25.4,27,18,19,1250,1300,55,60,2.16,2.74,2.27,2.64,2.13,2.62,2.23,2.52,81.27,102.32,86.05,97.65,78.298,104.153,85.345,97.158,2.29,2.73,2.39,2.68,2.45,2.74,2.53,2.67
19,5.5,7,23.8,25.39,17,18,1250,1300,55,60,2.04,2.75,2.26,2.52,1.95,2.64,2.12,2.55,82.02,104.01,86.87,98.83,81.133,102.246,86.438,97.419,,,,,,,,


In [13]:
template_df.to_csv('complete_ranges.csv',  index=False)